In [1]:
import pyclesperanto_prototype as cle

from skimage.io import imread, imsave

# initialize GPU
device = cle.select_device("GTX")
print("Used GPU: ", device)

# load data
image = imread('https://imagej.nih.gov/ij/images/blobs.gif')

# process the image
inverted = cle.subtract_image_from_scalar(image, scalar=255)
blurred = cle.gaussian_blur(inverted, sigma_x=1, sigma_y=1)
binary = cle.threshold_otsu(blurred)
labeled = cle.connected_components_labeling_box(binary)

# The maxmium intensity in a label image corresponds to the number of objects
num_labels = cle.maximum_of_all_pixels(labeled)

# print out result
print("Num objects in the image: " + str(num_labels))

# save image to disc
imsave("result.tif", labeled)

Used GPU:  <Apple M1 on Platform: Apple (2 refs)>
Num objects in the image: 62.0


/var/folders/vg/pkq1xs_x1b559nk61kjgcgkh0000gn/T/ipykernel_91044/2795466400.py:25: UserWarning: result.tif is a low contrast image
  imsave("result.tif", labeled)


In [2]:
import pyopencl as cl